In [2]:
# IMDB - 2 layer network - train + validation + test, 20 epochs (pyTorch 1.2 version)
# notes: 
# - uses keras to fetch the dataset for consistancy with other code samples, data operations and feeding to network done with numpy
# - accuracy for train/validation/test is measured by passing entire set through the network (eg. in tf 2.x it's mean of batch, also possible to do in pytorch)

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

from keras.datasets import imdb

#network parameters
n_input = 10000 #input size for a single sample (10000 words)

#hyperparamters
batch_size = 512
eta = 0.001 # keras default for rmsprop = 0.001
max_epoch = 20


# 1. get data (using keras to download for consistancy with other code samples)
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= n_input)

#pre-process data into tensors
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test =  vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

#validation set to use during training
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# 2. network architecture
class IMDBModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10000, 16)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)
        self.relu2 = nn.ReLU()
        self.out = nn.Linear(16, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        a2 = self.fc2(h1)
        h2 = self.relu2(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
network = IMDBModel()

# 3. select optimizer and loss

optimizer = optim.RMSprop(network.parameters(), lr= eta)
loss_criterion = nn.BCELoss()

# 4. train / run network

#define training procedure
def train_step(batch_x, batch_y):
    #zero the parameter gradients
    optimizer.zero_grad()
    #forward pass
    outputs = network(batch_x.float())
    loss = loss_criterion(outputs, batch_y)
    #backward pass
    loss.backward()
    #propagate/update weights
    optimizer.step()

#define accuracy
def accuracy(model_output, labels):
    output_mod = model_output.round().reshape(model_output.shape[0],)
    return (output_mod == labels).sum() / len(labels)
    
print("Training with {} epochs, {} batches each ... ".format(max_epoch, int(len(partial_x_train) / batch_size) ))
for epoch in range(max_epoch):    
    batch_steps = int(len(partial_x_train) / batch_size)
    for i in range(batch_steps):
        batch_x = torch.tensor(partial_x_train[i*batch_size:(i+1)*batch_size])
        batch_y = torch.tensor(partial_y_train[i*batch_size:(i+1)*batch_size])
        train_step(batch_x, batch_y)
    #check output for full train
    output_tr = network(torch.tensor(partial_x_train).float())
    loss_tr = loss_criterion(output_tr, torch.tensor(partial_y_train))
    acc_tr = accuracy(output_tr.detach().numpy(), partial_y_train)
    #check output for validate set
    output_val = network(torch.tensor(x_val).float())
    loss_val = loss_criterion(output_val, torch.tensor(y_val))
    acc_val = accuracy(output_val.detach().numpy(), y_val)
    print("Epoch {} - loss: {} - acc: {} - loss_val: {} - acc_val: {}".format(epoch+1,loss_tr, acc_tr, loss_val,acc_val))

# 5. test model
acc_tst = accuracy(network(torch.tensor(x_test).float()).detach().numpy(), y_test)
print("test_acc: {}".format(acc_tst))

Training with 20 epochs, 29 batches each ... 


C:\Users\adamw\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
C:\Users\adamw\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([15000])) that is different to the input size (torch.Size([15000, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
C:\Users\adamw\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reducti

Epoch 1 - loss: 0.4113349914550781 - acc: 0.9067333333333333 - loss_val: 0.4452159106731415 - acc_val: 0.8703
Epoch 2 - loss: 0.21281594038009644 - acc: 0.9392666666666667 - loss_val: 0.29257526993751526 - acc_val: 0.8903
Epoch 3 - loss: 0.15908195078372955 - acc: 0.9534 - loss_val: 0.27895647287368774 - acc_val: 0.8897
Epoch 4 - loss: 0.12361491471529007 - acc: 0.9659333333333333 - loss_val: 0.27667757868766785 - acc_val: 0.8902
Epoch 5 - loss: 0.10163246840238571 - acc: 0.974 - loss_val: 0.2920284569263458 - acc_val: 0.885
Epoch 6 - loss: 0.08374352008104324 - acc: 0.9792 - loss_val: 0.30922406911849976 - acc_val: 0.8826
Epoch 7 - loss: 0.06579062342643738 - acc: 0.9855333333333334 - loss_val: 0.3244576156139374 - acc_val: 0.8825
Epoch 8 - loss: 0.0579681396484375 - acc: 0.9868 - loss_val: 0.35587945580482483 - acc_val: 0.88
Epoch 9 - loss: 0.08667836338281631 - acc: 0.9672666666666667 - loss_val: 0.45202934741973877 - acc_val: 0.8615
Epoch 10 - loss: 0.03767857700586319 - acc: 0.993